In [1]:
import pandas as pd
import numpy as np

In [2]:
# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject=CS" # get SP 22 CS classes
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())


In [3]:
classes = data_json['data']['classes']

In [4]:
# CS 1110, Bracy, start time, end time, location, description, prereqs/coreqs, semester offered
df = []
missedClasses = 0

for subject in ["CS", "INFO", "ECE", "ECON", "ENGL", "PHIL", "AAS", "AEM", "AEP", "AGSCI", "AIIS", "AIRS",
               "AKKAD", "ALS", "AMST", "ANSC", "ANTHR", "BTRY", "BIOAP", "BIOEE", "BIOG", "BIOMG", "BIOMI",
               "BIOMS", "BIONB", "BME"]: #, "ECON"]"ENGL", "PHIL"]:

    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject={subject}"
    response = urlopen(url)

    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())

    classes = data_json['data']['classes']

    for class_ in classes:
        #print(class_)
        try:
            #if (int(class_['catalogNbr'])) > 5900:
            #    continue
            #print(class_)
            obj = class_['enrollGroups'][0]['classSections'][0]['meetings'][0]

            obj2 = obj['instructors'][0]
            #print(obj)

            startTime = obj['timeStart']
            endTime = obj['timeEnd']
            location = obj['facilityDescr']
            x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], obj2['firstName'].strip() + ' ' + obj2['lastName'].strip(), startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]
            df.append(x)
        except:
            missedClasses += 1
print(missedClasses)

41


In [5]:
df1 = pd.DataFrame(np.array(df))
df1.shape

(918, 10)

In [6]:
df1 = df1.rename(columns={0:"Dept", 1:"Number", 2:"Course_Name", 3:"Professor", 4:"Start_Time", 5:"End_Time", 6:"Location",7:"Description",8:"Pre/coreqs",9:"Offered"})

In [7]:
median_grades = pd.read_csv("Cleaned_Medians.csv")

In [8]:
median_grades

,Unnamed: 0,Dept + Number,Professor,Median Grade,Semester (Ex: SP21),# of Students,Notes,Semester
0,1,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640",0.0
1,424,INFO 2950,Wilkens,A-,FA21,217,NaN,0.0
2,422,INFO 2450,Nathan Matias,A-,FA21,258,Crosslisted with COMM 2450,0.0
3,347,GOVT 3112,Bateman,A-,FA21,39,NaN,0.0
4,420,INFO 1300,Harms,A,FA21,264,NaN,0.0
...,...,...,...,...,...,...,...,...
519,129,BTRY 4030,Giles,B+,FA18,64,NaN,6.0
520,462,MATH 1220,Benjimin,B+,FA18,10,NaN,6.0
521,140,CEE 4510,Richardson,A-,FA17,44,NaN,8.0
522,171,COML 1109,Various (FWS),A-,FA17,NaN,NaN,8.0


In [9]:
median_grades = median_grades[['Dept + Number', 'Median Grade', 'Semester (Ex: SP21)', 'Semester']]

In [11]:
df1["Dept + Number"] = df1["Dept"] + ' ' + df1["Number"]

In [12]:
df1['Dept + Number'].value_counts()

BIOMG 4880    1
AMST 3405     1
AEM 2220      1
BIOAP 4130    1
INFO 4998     1
             ..
BIOEE 1640    1
AEM 3040      1
CS 5121       1
CS 4121       1
AMST 1951     1
Name: Dept + Number, Length: 918, dtype: int64

In [13]:
df2 = df1.merge(median_grades, on='Dept + Number', how="outer")

In [14]:
df2 = df2.dropna(subset=['Dept'])

In [15]:
df2

,Dept,Number,Course_Name,Professor,Start_Time,End_Time,Location,Description,Pre/coreqs,Offered,Dept + Number,Median Grade,Semester (Ex: SP21),Semester
0,CS,1110,Introduction to Computing Using Python,Anne Bracy,09:05AM,09:55AM,Bailey Hall 101,Programming and problem solving using Python. ...,,"Fall, Spring, Summer.",CS 1110,B+,FA21,0.0
1,CS,1112,Introduction to Computing Using MATLAB,Kit-Yee Daisy Fan,02:40PM,03:30PM,Hollister Hall B14,Programming and problem solving using MATLAB. ...,"Corequisite: MATH 1110, MATH 1910, or equivalent.","Fall, Spring.",CS 1112,B,FA21,0.0
2,CS,1132,Short Course in MATLAB,Kit-Yee Daisy Fan,02:40PM,03:30PM,Phillips Hall 219,Introduction to the MATLAB programming languag...,,"Fall, Spring.",CS 1132,NaN,NaN,NaN
3,CS,1133,Short Course in Python,Craig Frey,12:25PM,01:15PM,Upson Hall 142,Introduction to the Python programming languag...,,"Fall, Spring.",CS 1133,NaN,NaN,NaN
4,CS,1300,Introductory Design and Programming for the Web,Royal Westwater,11:25AM,12:40PM,Olin Hall 165,The World Wide Web is both a technology and a ...,,"Fall, Spring.",CS 1300,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,BME,6410,Cell and Molecular Mechanobiology,Jan Lammerding,09:40AM,10:55AM,Weill Hall 224,Mechanobiology describes how cells and tissues...,"Prerequisite: BIOMG 1350 or equivalent, BME 30...",Spring.,BME 6410,NaN,NaN,NaN
914,BME,7020,Biomedical Engineering Research Seminar,Peter Doerschuk,09:05AM,09:55AM,Weill Hall 226,Designed to train graduate students in public ...,,"Fall, Spring.",BME 7020,NaN,NaN,NaN
915,BME,7030,Graduate Student Teaching Experience,Jan Lammerding,,,To Be Assigned,Guided individual experience in laboratory ins...,,"Fall, Spring.",BME 7030,NaN,NaN,NaN
916,BME,7130,Core Concepts in Disease,Nate Cira,11:25AM,12:40PM,Weill Hall 226,Most diseases emerge due to a relative small n...,Prerequisite: introductory biology.,Spring.,BME 7130,NaN,NaN,NaN


In [16]:
df2.to_csv("Classes_With_Medians.csv")